# KOSMOS予約
読みたい本のリストを慶応メディアセンターのKOSMOSで検索した結果をタブで開くまでを自動化

In [54]:
# coding: UTF-8

#ライブラリをインポート

import urllib.request

from bs4 import BeautifulSoup

import lxml.html

from selenium import webdriver

driver = webdriver.Chrome("/Users/keigo/pyworks/chromedriver")

import webbrowser

from time import sleep

## 検索したいbook_listを作成
- 選択肢は以下の3つ
    - 手動で入力
    - 静的なページのタグの情報から取得（amazonのジャンル別ランキングなど）
    - 動的なページのタグの情報からseleniumで取得（amazonのオールタイムベストなど）
- 拾うタグの指定方法
    - タグ名 & 属性値 (class, name, id) が確実
    - それで対処できない場合、CSSセレクタベタ打ち & 数字を for文で回すのもアリ

### 手動で入力

In [55]:
# 手動で入力

book_list = []  

string = '''入門ウェブ分析論
イシューからはじめよ
知的生産力が劇的に高まる最強フレームワーク100
論点思考
仮設思考
USJを劇的に変えたたった一つの考え方
売れるもマーケ 当たるもマーケ
シェアしたがる心理
「欲しい」の本質
行動デザインの教科書'''
book_list = string.splitlines()
print(book_list)

['入門ウェブ分析論', 'イシューからはじめよ', '知的生産力が劇的に高まる最強フレームワーク100', '論点思考', '仮設思考', 'USJを劇的に変えたたった一つの考え方', '売れるもマーケ 当たるもマーケ', 'シェアしたがる心理', '「欲しい」の本質', '行動デザインの教科書']


### 静的なページから取得

In [38]:
#静的なページのタグの情報からCSSセレクタで取得（amazonのジャンル別ランキングなど）

#amazonからの取得例
# url = "https://www.amazon.co.jp/gp/bestsellers/books/466298/ref=zg_bs_nav_b_1_b"
# for img in soup.find_all('img',class_="a-thumbnail-left"):
#       tags.append(img.get('alt'))    

# url = "https://www.amazon.co.jp/gp/bestsellers/books/500542/ref=zg_bs_nav_b_3_492262" #コンピュータミュージック

tag = 'img' #書籍画像のimageタグがあればそれが良い

#タグのclassがあれば入力、なければ上の階層に遡って探す
class_ = "a-thumbnail-left" #amazonトレンドランキング

tags = []

# URLにアクセスする htmlが帰ってくる 
html = urllib.request.urlopen(url)

# htmlをBeautifulSoupで扱う
soup = BeautifulSoup(html, "html.parser")




for img in soup.find_all(tag, class_ = class_):
    tags.append(img.get('alt'))


book_list = list(set(tags))

for title in book_list:
    print(title)

作りながら覚える 3日で作曲入門
エンジニア直伝! クリエイターのためのミックス&マスタリング最新テクニック (サウンド&レコーディング・マガジン)
達人と作る アナログシンセサイザー自作入門 改訂版2017
レコーディング/ミキシングの全知識 [改訂版] (「全知識」シリーズ)
シンセサイザーがわかる本 〜予備知識から歴史、方式、音の作り方まで
VOCALOID4 Editorでボーカロイドを思い通りに歌わせる本 付属ムービーで操作方法が一目瞭然! (DVD-ROM付)
リズム&ドラム・マガジン ドラム・パターン大事典326 全フレーズ映像対応! (DVD付き) (リットーミュージック・ムック)
DTMerのためのド派手なバンドアレンジがガンガン身に付く本
DTMミキシングの基礎技術を解説付きで実演しちゃいました (オーディオデータダウンロード対応)
クリプトン・フューチャー・メディア公認 初音ミク V4X 徹底攻略ガイドブック 調声からDAWでの曲作りまでわかる本 (データ・ダウンロード対応)
音圧アップのためのDTMミキシング入門講座! (DVD-ROM付)
できる ゼロからはじめるパソコン音楽制作超入門(はじめる前に観るDVD付) (できるシリーズ)
DJをはじめるための本 DJソフトを使って自宅で手軽にスタート
ミュージッククリエイターハンドブック MIDI検定公式ガイド
ボカロで覚える 中学数学 (MUSIC STUDY PROJECT)
サウンドエフェクトの作り方 (I・O BOOKS)
これが知りたかった! 音楽制作の秘密100 作曲/編曲/作詞からコンペ必勝法まで現役プロが明かすQ&A形式ノウハウ集
ボカロで覚える 中学英単語 (MUSIC STUDY PROJECT)
Cubase Pro 9ではじめるDTM&曲作り ビギナーが中級者になるまで使える操作ガイド+楽曲制作テクニック
かっこいいコード進行108 転調! テンション! ツーファイブ!  (SMFデータダウンロード対応)


### 動的なページから取得

In [5]:
#動的なページのタグの情報からseleniumのCSSセレクタで取得（amazonのオールタイムベストなど）
#オールタイムベストページの動的処理にseleniumで対応する

# amazonのオールタイムベストのURLを入れる
url = "https://www.amazon.co.jp/s/ref=s9_acsd_hps_bw_clnk_r?__mk_ja_JP=%83%4A%83%5E%83%4A%83%69&node=465392,!2141347051,!2141373051,3311704051,3332811051&search-alias=stripbooks&bbn=3311704051&pf_rd_m=AN1VRQENFRJN5&pf_rd_s=merchandised-search-5&pf_rd_r=9QPZ5WXGQGRKY7SGJQ3G&pf_rd_t=101&pf_rd_p=b18fc82b-4a40-4ba2-9fa5-8cd3d812f64a&pf_rd_i=3311704051"
url_2 = "https://www.amazon.co.jp/s/ref=sr_pg_2?rh=n%3A465392%2Cn%3A%212141347051%2Cn%3A%212141373051%2Cn%3A3311704051%2Cn%3A3332811051&page=2&bbn=3311704051&ie=UTF8&qid=1539769702"

#指定したURLに遷移する
driver.get(url)

book_list = []

n=0

#Googleの検索テキストボックスの要素をCSSセレクタ(クラスセレクタ)から指定
for n in range(100):
    try:
        t = driver.find_element_by_css_selector("#result_" + str(n)  +" > div > div:nth-child(3) > div:nth-child(1) > a > h2")
        book_list.append(t.text)
    except:
        pass

for book_title in book_list:
    print(book_title)

#いけた！！

ビジョナリー・カンパニー ― 時代を超える生存の原則
マネジメント[エッセンシャル版] - 基本と原則
ビジョナリー・カンパニー 2 - 飛躍の法則
ストーリーとしての競争戦略 ―優れた戦略の条件 (Hitotsubashi Business Review Books)
失敗の本質―日本軍の組織論的研究 (中公文庫)
プロフェッショナルマネジャー
稲盛和夫の実学―経営と会計
イノベーションのジレンマ (―技術革新が巨大企業を滅ぼすとき (Harvard business school press))
ザ・ゴール ― 企業の究極の目的とは何か
小倉昌男　経営学
戦略プロフェッショナル―シェア逆転の企業変革ドラマ (日経ビジネス人文庫)
ドラッカー名著集1　経営者の条件
はじめの一歩を踏み出そう―成功する人たちの起業術
トヨタ生産方式――脱規模の経営をめざして
アメーバ経営―ひとりひとりの社員が主役
戦略思考コンプリートブック
ひとりでも部下のいる人のための世界一シンプルなマネジメント術 3分間コーチ
スティーブ・ジョブズ I
競争の戦略
最高のリーダー、マネジャーがいつも考えているたったひとつのこと
知識創造企業
リッツ・カールトンが大切にする サービスを超える瞬間
日本でいちばん大切にしたい会社
企業戦略論【上】基本編 競争優位の構築と持続


## KOSMOSでタイトルを検索してURLのリストを取得

Broken pipe error が出ると検索が止まってしまう問題をなんとかしたい！

In [56]:
#KOSMOSでタイトルを検索してURLのリストを取得

url_list = []

for book_title in book_list:
    driver.get("http://kosmos.lib.keio.ac.jp/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=default_tab&indx=1&dum=true&srt=rank&vid=KEIO&frbg=&tb=t&vl%28freeText0%29=&vl%28489411217UI1%29=all_items&vl%281UIStartWith0%29=contains&vl%28213708702UI0%29=any&vl%28213708702UI0%29=title&vl%28213708702UI0%29=any&scp.scps=scope%3A%28KEIO%29%2Cprimo_central_multiple_fe")
    driver.find_element_by_id('search_field').send_keys(book_title)
    driver.find_element_by_id('goButton').click()
    #検索部分一致にしたい！
    try:
        driver.find_element_by_class_name('EXLResultTab').click()   #最上位の検索結果の"全ての所蔵"クリック
        url_list.append(driver.current_url)
        sleep(0.5)
    except:
        pass

driver.quit()

for url in url_list:
    print(url)

http://kosmos.lib.keio.ac.jp/primo_library/libweb/action/search.do;jsessionid=553223949D2CC641114BB66D01A51DCE?fn=search&ct=search&initialSearch=true&mode=Basic&tab=default_tab&indx=1&dum=true&srt=rank&vid=KEIO&frbg=&tb=t&vl%28freeText0%29=%E5%85%A5%E9%96%80%E3%82%A6%E3%82%A7%E3%83%96%E5%88%86%E6%9E%90%E8%AB%96&vl%28489411217UI1%29=all_items&vl%281UIStartWith0%29=contains&vl%28213708702UI0%29=any&vl%28213708702UI0%29=title&vl%28213708702UI0%29=any&scp.scps=scope%3A%28KEIO%29%2Cprimo_central_multiple_fe
http://kosmos.lib.keio.ac.jp/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=default_tab&indx=1&dum=true&srt=rank&vid=KEIO&frbg=&tb=t&vl%28freeText0%29=%E3%82%A4%E3%82%B7%E3%83%A5%E3%83%BC%E3%81%8B%E3%82%89%E3%81%AF%E3%81%98%E3%82%81%E3%82%88&vl%28489411217UI1%29=all_items&vl%281UIStartWith0%29=contains&vl%28213708702UI0%29=any&vl%28213708702UI0%29=title&vl%28213708702UI0%29=any&scp.scps=scope%3A%28KEIO%29%2Cprimo_central_multiple_fe
http://kos

## リストを確認してこのウィンドウで全てのurlを開く

In [57]:
#リストを確認してこのウィンドウで全てのurlを開く
#あらかじめKOSMOSでログインしておく



for url in url_list:
    webbrowser.open(url)

#内容を確認して手動で予約

## これからやりたいことメモ
- KOSMOS検索を部分一致にしたい（タイトルの余計な文言を削りたい or 形態素ごとにスペースを空けて対処したい）
- amazonのランキングをページ送りしたい
- class指定で取得した時に同じclassだと順番がバラバラになってしまう問題を解決したい
- seleniumをバックグラウンドで実行したい

# 以下試行錯誤の跡

In [17]:
#試行錯誤
#オールタイムベストから書籍タイトルのリストを取得したいが、動的に処理されているのでできないっぽい？


# amazonのランキングのURLを入れる
url = "https://www.amazon.co.jp/s/ref=s9_acsd_hps_bw_clnk_r?__mk_ja_JP=%83%4A%83%5E%83%4A%83%69&node=465392,!2141347051,!2141373051,3311704051,3332811051&search-alias=stripbooks&bbn=3311704051&pf_rd_m=AN1VRQENFRJN5&pf_rd_s=merchandised-search-5&pf_rd_r=9QPZ5WXGQGRKY7SGJQ3G&pf_rd_t=101&pf_rd_p=b18fc82b-4a40-4ba2-9fa5-8cd3d812f64a&pf_rd_i=3311704051"

# URLにアクセスする htmlが帰ってくる 
html = urllib.request.urlopen(url)
# htmlをBeautifulSoupで扱う
soup = BeautifulSoup(html, "html.parser")
print (soup.prettify())

# book_list = []

#CSSセレクタで書籍の画像を含むタグを取得
#Outer HTML
#<img src="https://images-fe.ssl-images-amazon.com/images/I/41j4+PGYDQL._AC_US200_.jpg" srcset="https://images-fe.ssl-images-amazon.com/images/I/41j4+PGYDQL._AC_US200_.jpg 1x, https://images-fe.ssl-images-amazon.com/images/I/41j4+PGYDQL._AC_US300_FMwebp_QL65_.jpg 1.5x, https://images-fe.ssl-images-amazon.com/images/I/41j4+PGYDQL._AC_US400_FMwebp_QL65_.jpg 2x, https://images-fe.ssl-images-amazon.com/images/I/41j4+PGYDQL._AC_US500_FMwebp_QL65_.jpg 2.5000x" width="200" height="200" alt="ビジョナリー・カンパニー ― 時代を超える生存の原則" class="s-access-image cfMarker" data-search-image-load="">
#この中からalt="ビジョナリー・カンパニー ― 時代を超える生存の原則"を取得する

print(soup.select('#result_0 > div > div.a-row.a-spacing-base > div > div > a > img'))



# ParserError: Document is empty、なぜ？
#Xpathで書籍タイトルを囲むタグを取得
#//*[@id="result_0"]/div/div[3]/div[1]/a/h2

# html = urllib.request.urlopen(url)
# soup = BeautifulSoup(html, "lxml")
# dom = lxml.html.fromstring(html.read())   # ParserError: Document is empty
# t = dom.xpath('//*[@id="result_0"]/div/div[3]/div[1]/a/h2') #xxxの部分をテキスト指定でhrefを取得

# print(t)



#空のリストが出力されて失敗、なぜ？

#CSSセレクタで書籍タイトルを囲むタグを取得
# book_title_tag = soup.select('#result_0 > div > div:nth-child(3) > div:nth-child(1) > a > h2')
# print(book_title_tag)


#繰り返し
# n = 0

# book_title_tag = soup.select('#result_' + n + '> div > div:nth-child(3) > div:nth-child(1) > a > h2')
# print(book_title_tag)

## n =+ 1


#タグの中の文字列（書籍タイトル）を取得
# bool_title = soup.book_title_tag.text

# print(book_tilte)



# print(book_list)
# book_list_uniq = list(set(book_list))

# print(book_list_uniq)


<!DOCTYPE doctype html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="ja-jp">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="//images-fe.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="//m.media-amazon.com" rel="dns-prefetch"/>
  <link href="//completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:feature:aui-assets -->
  <link href="https://images-fe.ssl-images-amazon.com/images/I/61GSlhxiM1L._RC|01evdoiemkL.css,01K+Ps1DeEL.css,31bAdTWQ3tL.css,11NK7V6nvxL.css,11pdJ7gGeEL.css,21LK7jaicML.css,11L58Qpo0GL.css,21EuGTxgpoL.css,01Xl9KigtzL.css,117-Dg7wwnL.css,21GwE3cR-yL.css,019SHZnt8RL.css,01gv-pABIRL.css,11vZhCgAHbL.css,21Mne54CsmL.css,11WgRxUdJRL.css,01dU8+SPlFL.css,11DGn6WmpTL.css,01SHjPML6tL.css,111-D2qRjiL.css,01QrWuRrZ-L.css,31VvBsfI8-L.css,114KWZGKCVL.css,01cbS3UK11L.css,21nb

In [23]:
#試行錯誤
#予約完了まで行きたいがポップアップウィンドウをどう操作すれば良いのかわからない

driver.get("http://kosmos.lib.keio.ac.jp/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=default_tab&indx=1&dum=true&srt=rank&vid=KEIO&frbg=&tb=t&vl%28freeText0%29=&vl%28489411217UI1%29=all_items&vl%281UIStartWith0%29=contains&vl%28213708702UI0%29=any&vl%28213708702UI0%29=title&vl%28213708702UI0%29=any&scp.scps=scope%3A%28KEIO%29%2Cprimo_central_multiple_fe")
driver.find_element_by_id('search_field').send_keys("あ")
driver.find_element_by_id('goButton').click()

#最上位の検索結果の"全ての所蔵"クリック
driver.find_element_by_class_name('EXLResultTab').click()

#"リクエスト"をクリック
#driver.find_elements_by_xpath("/*/script").click()
#td.td1

#javascriptだから動かない？
#driver.find_elements_by_link_text('全ての所蔵').click()
#driver.find_elements_by_link_text('リク

#受け取り地区に"日吉"がある場合選択

#"GO"をクリック